In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.family'] = 'Microsoft YaHei'
plt.rcParams['axes.unicode_minus'] = False

df_bookkeep = pd.read_csv("../data-source/201807-202207.csv")
df_bookkeep['交易时间'] = pd.to_datetime(df_bookkeep['交易时间'], format='%Y/%m/%d %H:%M:%S')
df_bookkeep['金额(元)'] = pd.to_numeric(df_bookkeep['金额(元)'])

df_bookkeep['月度'] = df_bookkeep['交易时间'].dt.strftime('%Y-%m')

def process_amount(row):
    amount = row['金额(元)']
    if row['收/支'] == '收入':
        return amount
    else:
        return -amount

df_bookkeep['开销'] = df_bookkeep.apply(process_amount, axis=1)

df_bookkeep.head()


In [ ]:
table_payment = pd.pivot_table(df_bookkeep, index='月度', columns='类型', values='开销', aggfunc='sum')
table_payment.fillna(0, inplace=True)

In [ ]:
table_payment["月结余"] = table_payment.sum(axis=1)
table_payment["月支出"] = table_payment["月结余"] - table_payment['工资'] - table_payment['交易']

table_payment

In [ ]:
# 定义应用于总支出列的样式函数
def color_render(val):
    colors = np.where(val >= -5000, 'color: green', np.where(val <= -7000, 'color: red', ''))
    return colors

table_styled = table_payment.style.apply(color_render, subset=["月支出"])

In [ ]:
plt.figure(figsize=(20, 10))
type_result = -(table_payment.drop(["工资","月结余", "月支出", "交易"], axis=1).sum(axis=0))

type_result.plot(kind="bar")

In [ ]:
plt.figure(figsize=(30, 10))
monthly_result = abs(table_payment["月支出"])

monthly_result.plot(kind="line")